# Chapter 2.1 - Advanced Qbit

In this section of chapter 2 we will be walking through how to use Qbit variables in more complex expressions and assignments by chaining multiple operations together. We will also see how to analyze real world problems and break them down into an expression that can be solved by the Dwave quantum computer.

To start we will import the Qbit class from dann5 and create our complex expressions:

In [57]:
from dann5.d5 import Qbit

a0 = Qbit("a0", 1) 
a1 = Qbit("a1")
a2 = Qbit("a2")
a3 = Qbit("a3", 0)
a4 = Qbit("a4")

print(a0)
print(a1)
print(a2)
print(a3)
print(a4)

a0\1\
a1\S\
a2\S\
a3\0\
a4\S\


Within the dann5 library there is a distinct difference between binary bitwise operators (i.e. xor, and, or) and quantum comparison (i.e. ==, !=, <, >, ~) operators. 

When we see quantum comparison operations we can think of them as rules for the sets of possibilities given, we are creating a relationship between multiple quantum variables that will be enforced in every solution presented. For instance the following code symbolizes that the XOR operation between variable a2 and variable a3 must always be equal to the value of a4.

> ((a2\S\ ^ a3\0\) == a4\S\))

In [58]:
expression = (a0 & a1) | ((a2 ^ a3) == a4)
print(expression)

((a0\1\ & a1\S\) | ((a2\S\ ^ a3\0\) == a4\S\))


When using the decomposed line output function the first line in the output is always from outermost operation (top) to innermost (down). Priority of operators is enforced by the syntax, which is the same as in classical computing, in this case sinse we are using Python the priority of operators in Python determines the order. So the top most output is for OR operation of the two sub expressions:

> _|8\S\ = _&16\S\ **|** _^20\S\;

For the third line in the output we see the rule that we created in the expression, this is showing the above definition of the following code: 

> (a2\S\ ^ a3\0\) == a4\S\

So now when it is printed in the decomposed version the code generates a variable _^0\S\ to symolize the operation between a2 and a3 variable, as shown below: 

> _^0\S\ == a4\S\;


In [59]:
print(expression, "\n")
print(expression.toString(True).replace(";", ";\n"))

((a0\1\ & a1\S\) | ((a2\S\ ^ a3\0\) == a4\S\)) 

_|15\S\ = _&23\S\ | _^27\S\;
 _&23\S\ = a0\1\ & a1\S\;
 _^27\S\ == a4\S\;
 _^27\S\ = a2\S\ ^ a3\0\;
 


Now if we examine the first line in the decomposed logic we will see that the variable  _^0\S\ is used in the first line next to the OR operator. This is due to the fact that since  _^0\S\ and the a4 variable must have the same value we can interchangeably use them in that place.

The code however does not anticipate which variable you would like to pass to the higher level in the decomposed logic. Simply it takes the left variable in the == comparison operation and passes that variable to the higher level operation, as shown in the below code if you switch them around you will now see a4 variable in the first line:

> _|1\S\ = _&3\S\ | **a4\S\**; 

**Note**: make sure to double check the number next to the operator symbol for each generated variable, it will increase so that every variable is unique. This is especially important when using two or more of the same operator in an expression

**Note**: Bitwise operations work as above OR operation (XOR, NOR, AND, NAND & NXOR) and comparison operations work similar to == above (!=, <, >, <=, >=). 

**Note** Tilda function (~) is unique as it is a bitwise operation but it functions similar to comparison operations.

In [60]:
print(expression, "\n")
print(expression.toString(True).replace(";", ";\n"))

expression1 = (a0 & a1) | (a4 == (a2 ^ a3))
print(expression1, "\n")
print(expression1.toString(True).replace(";", ";\n"))

((a0\1\ & a1\S\) | ((a2\S\ ^ a3\0\) == a4\S\)) 

_|15\S\ = _&23\S\ | _^27\S\;
 _&23\S\ = a0\1\ & a1\S\;
 _^27\S\ == a4\S\;
 _^27\S\ = a2\S\ ^ a3\0\;
 
((a0\1\ & a1\S\) | (a4\S\ == (a2\S\ ^ a3\0\))) 

_|16\S\ = _&24\S\ | a4\S\;
 _&24\S\ = a0\1\ & a1\S\;
 a4\S\ == _^28\S\;
 _^28\S\ = a2\S\ ^ a3\0\;
 


All variable values will be listed above as well as intermediary generated values. The line below is a one possible solution for the values, you can double check that all the decomposed expressions work with these values:

> _|0\0\;  _&0\0\;  a0\1\;  a1\0\;  _^0\0\;  a2\0\;  a3\0\;  a4\0\
>
> ((a0\1\ & a1\S\) | (a4\S\ == (a2\S\ ^ a3\0\)))

Now we can import the solver and solve the complex expression. The most important variables to look at are the a1, a2, a3, a4 and _|3. You can then grab the values for these variables for each solution and insert them into the expression or the decomposed expression to see that they are all correct:

In [61]:
from dann5.dwave import Solver 
Solver.Active() 
print(expression.solve())

_|15\0\; _&23\0\; a0\1\; a1\0\; _^27\0\; a2\0\; a3\0\; a4\0\
_|15\1\; _&23\0\; a0\1\; a1\0\; _^27\1\; a2\1\; a3\0\; a4\1\
_|15\1\; _&23\1\; a0\1\; a1\1\; _^27\0\; a2\0\; a3\0\; a4\0\
_|15\1\; _&23\1\; a0\1\; a1\1\; _^27\1\; a2\1\; a3\0\; a4\1\



Here we are solving the secondary expression we created where we flipped the left side and right side of the equal (==) comparison operation. As we can see by analyzing the solutions, the values are the same, only the order to which the variables are outputted from left to right has changed:

In [62]:
print(expression1.solve())

_|16\0\; _&24\0\; a0\1\; a1\0\; a4\0\; _^28\0\; a2\0\; a3\0\
_|16\1\; _&24\0\; a0\1\; a1\0\; a4\1\; _^28\1\; a2\1\; a3\0\
_|16\1\; _&24\1\; a0\1\; a1\1\; a4\0\; _^28\0\; a2\0\; a3\0\
_|16\1\; _&24\1\; a0\1\; a1\1\; a4\1\; _^28\1\; a2\1\; a3\0\



Now we can limit the scope of the solutions by setting an assignment. In the code below we will now request all solutions that have a final value of 1 for the entire expression:

In [63]:
ar = Qbit("ar",1)
qAssign = ar._((a0 & a1) | ((a2 ^ a3) == a4))

After printing the decomposed version of the assignment we see that it is exacty the same as the decomposed output for the expression:

In [65]:
print("\n {} \n\n {}\n".format(qAssign, 
                                   qAssign.toString(True).replace(";", ";\n")))


 ar\1\ = ((a0\1\ & a1\S\) | ((a2\S\ ^ a3\0\) == a4\S\)) 

 ar\1\ = _&25\S\ | _^29\S\;
 _&25\S\ = a0\1\ & a1\S\;
 _^29\S\ == a4\S\;
 _^29\S\ = a2\S\ ^ a3\0\;
 



And now we can solve the assignment and see that we do not get all 4 possible outputs, but only the 3 that had a final value of 1: 

In [20]:
print(qAssign.solve())

ar\1\; _&11\0\; a0\1\; a1\0\; _^14\1\; a2\1\; a3\0\; a4\1\
ar\1\; _&11\1\; a0\1\; a1\1\; _^14\0\; a2\0\; a3\0\; a4\0\
ar\1\; _&11\1\; a0\1\; a1\1\; _^14\1\; a2\1\; a3\0\; a4\1\



In [ ]:
expression = (a0 & a1) | ((a2 ^ a3) == a4)
print(expression)

# Real World Problem

Now we are going to see how we can generate a quantum expression to solve a real problem. The following problem is called "Friends and Enemies": 

Given a set of people and information about some pairs of people that are friends and some pairs that are enemies…..

How do we write a program that finds a solution in which all the friend and enemy relationships are true?

There are 2 tribes the 0 tribe and the 1 tribe, the 0 tribe are always enemies with the 1 tribe, where can all of these people belong so that all their individual relationships make sense. 


X (Xavier): 0
Y (Yolanda): 0  
Z (Zeke): 1
W (Wanda): 1

X & Y and W & Z: Friends (same qubit value)

Y & Z: Enemies (different qubit value)

This is a valid solution.

Can the D-Wave system find this solution for us? 

To start we need to define all the people that belong to either tribe. That we do by setting the Qbit variables: w, x, y and z.

In [66]:
x = Qbit("xaviar")
y = Qbit("yolanda")
z = Qbit("zeke")
w = Qbit("wanda")
_1 = Qbit("_1", 1)

print(w)
print(x)
print(y)
print(z)

wanda\S\
xaviar\S\
yolanda\S\
zeke\S\


The key peices of information to create an expression for any scenario are the following:

1) That XOR: "**^**" is equal to an **"unlike"** function, so if you want two variables to be enemies use this. 

2) That NXOR: ~(var1 ^ var2) or var1.nxor(var2) is equal to a **"like"** function. So if you want two variables to be friends use this

3) The expression includes all possible versions, so that includes versions where all the relationships are not true, these equal to a result of 0 (which can be thought of as false)

4) So by assigning all the possible solutions to 1 (can be thought of as True) we will get all the possibilities where the relationships are true 

With the following two rules in mind we can now write out the above scenario, where the following relationships exist: 

X & Y and W & Z: Friends

Y & Z: Enemies

In [67]:
fEAssignment = _1._(~(x ^ y) & ((y ^ z) & ~(z ^ w)));

# Same as:
# _1.assign(x.nxor(y) & ((y ^ z) & z.nxor(w)))
# _1.assign(x.alike(y) & y.unlike(z) & z.alike(w))

print(fEAssignment)
print()
print(fEAssignment.toString(True).replace(";", ";\n"))

_1\1\ = ((~_^30\S\ ~ (xaviar\S\ ^ yolanda\S\)) & ((yolanda\S\ ^ zeke\S\) & (~_^32\S\ ~ (zeke\S\ ^ wanda\S\))))

_1\1\ = ~_^30\S\ & _&26\S\;
 ~_^30\S\ ~ _^30\S\;
 _^30\S\ = xaviar\S\ ^ yolanda\S\;
 _&26\S\ = _^31\S\ & ~_^32\S\;
 _^31\S\ = yolanda\S\ ^ zeke\S\;
 ~_^32\S\ ~ _^32\S\;
 _^32\S\ = zeke\S\ ^ wanda\S\;
 


**Note**: In the decomposed output above we see a invert rule declared in a few lines that means that the element on the left side of the ~ operator has to be the invert of the variable on the right side of the operator

**Note**: The above expression is most optimal, we can add OR operator to list all possible scenarios where x and y are enemies to z and w, but that will add more nodes to the equation and cause it to run longer on simulator, without changing the logic

So we can write any expression with these 4 people where some are friends and some are enemies. And the quantum computing simulator will come up with all the possibilities where all of the relationships are true. 

Now to solve the assignment we have defined above we will use a slightly different method:

We will import the Qbinder class and use it to display the solutions to the assignment defined above.

We will start by defining a qbinder variable and passing the Qbit variables into it usign the "**<<**" symbol:

> Qbinder_var_name = Qbinder() << var1 << var2 << var3 ... 

We will then use the compute() function instead of solve to get the solutions (in this case called evaluations) and the compute function will return a list of possible solutions that we can iterate over and print out:

In [70]:
from dann5.d5 import Qbinder

qbinder = Qbinder() << x << y << z << w

evaluations = fEAssignment.compute()
print(['{}\n'.format(evaluation) for evaluation in evaluations])

['#[_^30]: 0; #[_^31]: 0; #[_^32]: 1; _&26: 1; _^30: 0; _^31: 1; _^32: 0; wanda: 1; xaviar: 0; yolanda: 0; zeke: 1; ~_^30: 1; ~_^32: 1; \n', '#[_^30]: 1; #[_^31]: 0; #[_^32]: 0; _&26: 1; _^30: 0; _^31: 1; _^32: 0; wanda: 0; xaviar: 1; yolanda: 1; zeke: 0; ~_^30: 1; ~_^32: 1; \n']


Finally we will add the list of solutions (evaluations variable) to the qbinder variable and print. 

In the below output for the solutions we see only the relevant variables that we defined at the beginning we see there are two possible solutions to the constraints we imposed at the start of the problem. To refresh here were the conditions: 

X & Y and W & Z: Friends

Y & Z: Enemies

And we know that anyone belonging to tribe 0 cannot be friends with anyone from tribe 1. And all members of a tribe are friends. So we see that in the two solutions, both conditions are satisfied. Taking the first as an example:

> xaviar\0\ yolanda\0\ zeke\1\ wanda\1\ 

Xaviar and Yolanda are in Tribe 0 so they are friends. Wanda and Zeke are in tribe 1 so they are friends. And Yolanda and Zeke are in two different tribes so they are enemies.

In [72]:
qbinder.add(evaluations);
print(qbinder)

xaviar\S\ yolanda\S\ zeke\S\ wanda\S\ {
xaviar\0\ yolanda\0\ zeke\1\ wanda\1\ 
xaviar\1\ yolanda\1\ zeke\0\ wanda\0\ 
xaviar\0\ yolanda\0\ zeke\1\ wanda\1\ 
xaviar\1\ yolanda\1\ zeke\0\ wanda\0\ 
}


One more concept to cover are the other comparison operators and how they work slightly differently to the equal (==) comparison operator. 

So in this example we have pretty much the same expression as the one covered at the top of this section. However, we are now using the less than (<) comparison operator instead.

Now when we see the decomposed version we see that the left side variable in the following line is being passed up to the higher level operation:

> Line 3: 
>  **_^18\S\** < a4\S\\;

> Line 1:
> _|6\S\ = _&14\S\ | _^18\S\\;

In [44]:
expression2 = (a0 & a1) | ((a2 ^ a3) < a4)
print(expression2, "\n")
print(expression2.toString(True).replace(";", ";\n"))
print(expression2.solve())

((a0\1\ & a1\S\) | ((a2\S\ ^ a3\0\) < a4\S\)) 

_|6\S\ = _&14\S\ | _^18\S\;
 _&14\S\ = a0\1\ & a1\S\;
 _^18\S\ < a4\S\;
 _^18\S\ = a2\S\ ^ a3\0\;
 
_|6\0\; _&14\0\; a0\1\; a1\0\; _^18\0\; a2\0\; a3\0\; a4\1\
_|6\1\; _&14\1\; a0\1\; a1\1\; _^18\0\; a2\0\; a3\0\; a4\1\



We also see here that now when we switch the a4 variable to the left side of the less than (<) operator. We now get different results: 

> _|7\0\; _&15\0\; a0\1\; a1\0\; **a4\0\;** _^19\1\; **a2\1\;** a3\0\

v.s.

> _|6\0\; _&14\0\; a0\1\; a1\0\; _^18\0\; **a2\0\;** a3\0\; **a4\1\**

This is due to the fact that we fundamentally changed the higher level operation by switching the place in the comparison operation rule. This is true with any comparison operator that is not equal to (==). So be careful when writing these complex equations with comparison operators.

Always remember that the left side of the comparison operator is what will be passed along to higher levels in an expression.

In [45]:
expression3 = (a0 & a1) | (a4 < (a2 ^ a3))
print(expression3, "\n")
print(expression3.toString(True).replace(";", ";\n"))
print(expression3.solve())

((a0\1\ & a1\S\) | (a4\S\ < (a2\S\ ^ a3\0\))) 

_|7\S\ = _&15\S\ | a4\S\;
 _&15\S\ = a0\1\ & a1\S\;
 a4\S\ < _^19\S\;
 _^19\S\ = a2\S\ ^ a3\0\;
 
_|7\0\; _&15\0\; a0\1\; a1\0\; a4\0\; _^19\1\; a2\1\; a3\0\
_|7\1\; _&15\1\; a0\1\; a1\1\; a4\0\; _^19\1\; a2\1\; a3\0\

